## Deliverable 2. Create a Customer Travel Destinations Map.

In [63]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,44.08,96,100,28.14,light rain
1,1,Mogadishu,SO,2.0371,45.3438,75.61,83,40,21.36,light rain
2,2,Thompson,CA,55.7435,-97.8558,59.16,88,75,8.05,broken clouds
3,3,Bambous Virieux,MU,-20.3428,57.7575,69.22,88,40,6.91,light rain
4,4,Bereda,ES,43.2686,-7.5406,61.57,91,79,6.26,broken clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [55]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mogadishu,SO,2.0371,45.3438,75.61,83,40,21.36,light rain
5,5,Rikitea,PF,-23.1203,-134.9692,75.34,74,83,12.82,broken clouds
6,6,Sao Joao Da Barra,BR,-21.6403,-41.0511,75.29,85,31,10.98,scattered clouds
7,7,Puerto Ayora,EC,-0.7393,-90.3518,76.96,89,99,8.01,overcast clouds
13,13,Albany,US,42.6001,-73.9662,78.40,51,100,9.10,overcast clouds


In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,75.61,light rain,2.0371,45.3438,
5,Rikitea,PF,75.34,broken clouds,-23.1203,-134.9692,
6,Sao Joao Da Barra,BR,75.29,scattered clouds,-21.6403,-41.0511,
7,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
13,Albany,US,78.40,overcast clouds,42.6001,-73.9662,
14,Torbay,CA,74.17,light intensity shower rain,47.6666,-52.7314,
16,Shimoda,JP,77.11,broken clouds,34.6667,138.9500,
19,Kousseri,CM,79.23,light rain,12.0769,15.0306,
20,Porto Walter,BR,76.84,overcast clouds,-8.2686,-72.7439,
25,Malakal,SS,75.04,overcast clouds,9.5334,31.6605,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found...skipping")  
hotel_df.head(10)

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,75.61,light rain,2.0371,45.3438,Hotel Juba
5,Rikitea,PF,75.34,broken clouds,-23.1203,-134.9692,People ThankYou
6,Sao Joao Da Barra,BR,75.29,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
7,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Albany,US,78.40,overcast clouds,42.6001,-73.9662,
14,Torbay,CA,74.17,light intensity shower rain,47.6666,-52.7314,Gallows Cove
16,Shimoda,JP,77.11,broken clouds,34.6667,138.9500,Shimoda Tokyu Hotel
19,Kousseri,CM,79.23,light rain,12.0769,15.0306,Le Pilotis
20,Porto Walter,BR,76.84,overcast clouds,-8.2686,-72.7439,Hotel Silva
25,Malakal,SS,75.04,overcast clouds,9.5334,31.6605,Nile Palace Hotel


In [57]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,75.61,light rain,2.0371,45.3438,Hotel Juba
5,Rikitea,PF,75.34,broken clouds,-23.1203,-134.9692,People ThankYou
6,Sao Joao Da Barra,BR,75.29,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
7,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Torbay,CA,74.17,light intensity shower rain,47.6666,-52.7314,Gallows Cove
16,Shimoda,JP,77.11,broken clouds,34.6667,138.9500,Shimoda Tokyu Hotel
19,Kousseri,CM,79.23,light rain,12.0769,15.0306,Le Pilotis
20,Porto Walter,BR,76.84,overcast clouds,-8.2686,-72.7439,Hotel Silva
25,Malakal,SS,75.04,overcast clouds,9.5334,31.6605,Nile Palace Hotel
28,Georgetown,MY,81.90,thunderstorm,5.4112,100.3354,Cititel Penang


In [59]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [60]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))